In [3]:
# ⭐️ 2号
%run run_softmax_ner.py\
--data_dir 'pretrained_data/task_core_aux_cond/k-fold/combine/2/'\
--model_type "bert"\
--model_name_or_path "bert-base-cased"\
--output_dir "pretrained_model/task_core_aux_cond/2/"\
--labels "pretrained_data/task_core_aux_cond/labels.txt"\
--overwrite_output_dir\
--do_eval\
--per_gpu_train_batch_size 16\
--per_gpu_eval_batch_size 16

# 验证成功，184条，结果可复现
"""
report =            precision    recall  f1-score   support

     Core    0.68724   0.67886   0.68303       246
      Aux    0.50000   0.51163   0.50575        43
     Cond    0.75000   0.75000   0.75000        16

micro avg    0.66337   0.65902   0.66118       305
macro avg    0.66414   0.65902   0.66155       305
"""

01/27/2022 14:07:22 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
01/27/2022 14:07:24 - INFO - __main__ -   Tokenizer arguments: {'do_lower_case': False}
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertSoftmaxForNer: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertSoftmaxForNer from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertSoftmaxForNer from the checkpoint of a model that you expect to be exactly identical (in

In [1]:
# 验证直出结果与测试结果一致
from infer_task import predict_task as bert_task
from metric import classification_report
from utils.utils_metrics import get_entities_bio


def get_bert_task_conll():
    return bert_task(data=None)


bert_task_pred_list, bert_task_true_list, bert_task_matrix, bert_task_tokens = get_bert_task_conll()
true_entities = get_entities_bio(bert_task_true_list)
pred_entities = get_entities_bio(bert_task_pred_list)

print(classification_report(true_entities, pred_entities))

"""
           precision    recall  f1-score   support

     Core    0.68724   0.67886   0.68303       246
      Aux    0.50000   0.51163   0.50575        43
     Cond    0.75000   0.75000   0.75000        16

micro avg    0.66337   0.65902   0.66118       305
macro avg    0.66414   0.65902   0.66155       305
"""

Evaluating: 100%|██████████| 12/12 [00:54<00:00,  4.53s/it]

           precision    recall  f1-score   support

     Core    0.68724   0.67886   0.68303       246
      Aux    0.50000   0.51163   0.50575        43
     Cond    0.75000   0.75000   0.75000        16

micro avg    0.66337   0.65902   0.66118       305
macro avg    0.66414   0.65902   0.66155       305




/Users/froyo/PycharmProjects/Infer_Pipeline/infer_task.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  res = softmax(torch.from_numpy(preds[i, j, :]))


In [21]:
# 使用train数据集替换dev数据集
# 预期：证明模型在已训练数据上的拟合，确定只用验证集
from infer_task import predict_task as bert_task
from metric import classification_report
from utils.utils_metrics import get_entities_bio


def get_bert_task_conll():
    return bert_task(data=None)


bert_task_pred_list, bert_task_true_list, bert_task_matrix, bert_task_tokens = get_bert_task_conll()
true_entities = get_entities_bio(bert_task_true_list)
pred_entities = get_entities_bio(bert_task_pred_list)

print(classification_report(true_entities, pred_entities))

"""
           precision    recall  f1-score   support

     Core    0.97420   0.97768   0.97594      1120
      Aux    0.94397   0.90496   0.92405       242
     Cond    0.95946   0.94667   0.95302        75

micro avg    0.96853   0.96381   0.96617      1437
macro avg    0.96834   0.96381   0.96600      1437
"""

Evaluating: 100%|██████████| 53/53 [03:59<00:00,  4.53s/it]
/Users/froyo/PycharmProjects/Infer_Pipeline/infer_task.py:147: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  res = softmax(torch.from_numpy(preds[i, j, :]))


           precision    recall  f1-score   support

     Core    0.97420   0.97768   0.97594      1120
      Aux    0.94397   0.90496   0.92405       242
     Cond    0.95946   0.94667   0.95302        75

micro avg    0.96853   0.96381   0.96617      1437
macro avg    0.96834   0.96381   0.96600      1437



'\n           precision    recall  f1-score   support\n\n     Core    0.68724   0.67886   0.68303       246\n      Aux    0.50000   0.51163   0.50575        43\n     Cond    0.75000   0.75000   0.75000        16\n\nmicro avg    0.66337   0.65902   0.66118       305\nmacro avg    0.66414   0.65902   0.66155       305\n'

In [19]:
# 脚手架，BERT评估打印差异信息

import json
import spacy
import en_core_web_lg
from tqdm import tqdm
from spacy import displacy
from spacy.tokens import Span
import spacy_alignments as tokenizations
from utils.utils_ner import get_entities
from utils.utils_metrics import get_entities_bio
from infer_task import predict_task as bert_task

DEV_BASE = 'pretrained_data/task_core_aux_cond/k-fold/combine/2/'
nlp = en_core_web_lg.load()

def get_bert_task_conll():
    return bert_task(data=None)

bert_task_pred_list, bert_task_true_list, bert_task_matrix, bert_task_tokens = get_bert_task_conll()

with open(DEV_BASE + 'dev.jsonl', 'r') as jsonl:
    lines = jsonl.readlines()

for idx, line in enumerate(tqdm(lines)):
    a = json.loads(line)
    res1 = nlp(a['text'])
    res2 = nlp(a['text'])
    tokens = [i.text for i in res1]
    print(bert_task_tokens[idx])
    print(tokens)
    print()
    b2v, v2b = tokenizations.get_alignments(bert_task_tokens[idx], tokens)
    pred_ents = []
    true_ents = []
    true_entities = get_entities_bio(bert_task_true_list[idx])
    pred_entities = get_entities_bio(bert_task_pred_list[idx])

#     if true_entities != pred_entities:

    for ent in pred_entities:
        typ, b, e = ent
        b_hat, e_hat = b2v[b][0], b2v[e][-1]
        pred_ents.append(Span(res1, b_hat, e_hat + 1, typ))

    for ent in true_entities:
        typ, b, e = ent
        b_hat, e_hat = b2v[b][0], b2v[e][-1]
        true_ents.append(Span(res2, b_hat, e_hat + 1, typ))

    res1.ents = pred_ents
    res2.ents = true_ents
    displacy.render(res1, style="ent", jupyter=True)
    displacy.render(res2, style="ent", jupyter=True)

  0%|          | 0/183 [00:00<?, ?it/s]

['Inter', 'with', 'the', 'courts', 'during', 'the', 'prosecution', 'of', 'cases', 'is', 'an', 'integral', 'part', 'of', 'the', 'responsibilities', 'of', 'police', 'personnel', '.']
['Interfacing', 'with', 'the', 'courts', 'during', 'the', 'prosecution', 'of', 'cases', 'is', 'an', 'integral', 'part', 'of', 'the', 'responsibilities', 'of', 'police', 'personnel', '.']



  1%|          | 1/183 [00:00<02:16,  1.33it/s]

['It', 'also', 'gives', 'the', 'user', 'the', 'ability', 'to', 'custom', 'the', 'results', 'view', 'by', 'criminal', '/', 'accused', 'or', 'by', 'cases', '.']
['It', 'also', 'gives', 'the', 'user', 'the', 'ability', 'to', 'customize', 'the', 'results', 'view', 'by', 'criminal', '/', 'accused', 'or', 'by', 'cases', '.']



  1%|          | 2/183 [00:00<01:15,  2.40it/s]

...

 99%|█████████▉| 181/183 [00:34<00:00,  5.15it/s]

['Professor', 'will', 'be', 'able', 'to', 'distribute', 'documents', 'to', 'students', '(', 'example', 's', ')', 'with', 'the', 'ability', 'to', 'lock', 'the', 'editing', 'tools', ',', 'so', 'un', 'change', 'to', 'the', 'document', 'will', 'be', 'impossible', '.']
['Professor', 'will', 'be', 'able', 'to', 'distribute', 'documents', 'to', 'students', '(', 'example', 'syllabus', ')', 'with', 'the', 'ability', 'to', 'lock', 'the', 'editing', 'tools', ',', 'so', 'unauthorized', 'change', 'to', 'the', 'document', 'will', 'be', 'impossible', '.']



 99%|█████████▉| 182/183 [00:34<00:00,  5.10it/s]

['The', 'user', 'should', 'periodically', 'change', 'the', 'password', 'currently', 'the', 'user', 'policy', 'requires', 'a', 'mandatory', 'password', 'change', 'every', 'three', 'months', '.']
['The', 'user', 'should', 'periodically', 'change', 'the', 'password', ';', 'currently', 'the', 'user', 'policy', 'requires', 'a', 'mandatory', 'password', 'change', 'every', 'three', 'months', '.']



100%|██████████| 183/183 [00:34<00:00,  5.30it/s]


In [17]:
# 脚手架，BERT推理打印结果

SENT = "A staging area using Cyclic Redundancy Check (CRCs) or checksums associated with each data file."

import spacy
import en_core_web_lg
from spacy import displacy
from spacy.tokens import Span
import spacy_alignments as tokenizations
from utils.utils_metrics import get_entities_bio
from infer_task import predict_task as bert_task

nlp = en_core_web_lg.load()

def get_bert_task(sent):
    data = [{'sent': sent}]
    return bert_task(data)


bert_task_pred_list, bert_task_true_list, bert_task_matrix, bert_task_tokens = get_bert_task(SENT)


res = nlp(SENT)
tokens = [i.text for i in res]
print(bert_task_tokens[0])
print(tokens)
print()
b2v, v2b = tokenizations.get_alignments(bert_task_tokens[0], tokens)
pred_ents = []
pred_entities = get_entities_bio(bert_task_pred_list[0])


for ent in pred_entities:
    typ, b, e = ent
    b_hat, e_hat = b2v[b][0], b2v[e][-1]
    pred_ents.append(Span(res, b_hat, e_hat + 1, typ))

res.ents = pred_ents
displacy.render(res, style="ent", jupyter=True)

Evaluating: 100%|██████████| 1/1 [00:02<00:00,  2.08s/it]

['A', 'staging', 'area', 'using', 'Cy', 'Red', 'Check', '(', 'CR', ')', 'or', 'checks', 'associated', 'with', 'each', 'data', 'file', '.']
['A', 'staging', 'area', 'using', 'Cyclic', 'Redundancy', 'Check', '(', 'CRCs', ')', 'or', 'checksums', 'associated', 'with', 'each', 'data', 'file', '.']

